In [1]:
import time
import os
import cv2

from lib.file.TIF import *
from lib.analysis.ROI_identification import *
from lib.file.ROI_writer import ROIFileWriter
from lib.file.ROI_reader import ROIFileReader
from lib.file.TSV import RegionExporter
from lib.analysis.align import ImageAlign
from lib.file.DAT import TracesDAT
from lib.trace import Tracer
from lib.utilities import *
from lib.analysis.laminar_dist import *
from lib.analysis.grid_latency import *

####### GRID ROIS #######
# for each recording:
## 1) divide the entire frame of recording into grid of square n_sq x n_sq ROIs
## 2) write these grid square ROIs to file
## 3) take SNRs and latencies to .dat file of each ROI from PhotoZ
## filter out no-response grid squares by SNR cutoff (or manual inspection?)
## write out output file of (ROI center x, ROI center y, SNR, latency)
## analysis: directed graph analysis of latency, 3-D plot of latency, 3-D plot of SNR

In [2]:
data_dir = "C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/mm_hidden/"

n_sq = 20  # number of pixels of each square edge in the grid.
max_num_rois = 100  # SNR cutoff 
latency_tolerance = 0.35  # ms tolerance to be counted simultaneous

# sets of files in each slice directory -- naming convention
# input
corners_file_prefix_layer = 'corners_layer'  # defines axes of L4 boundaries
electrode_file_prefix = 'electrode'
# also needed: 01_01_01_snr.dat

# output files
rois_file_prefix = 'sq_rois_' + str(n_sq) + "x" + str(n_sq) 
rois_centers_filenames = "roi_centers.txt"

image_data = {}
n_plots = 5000  # show up to how many plots

disable_photoZ_interact = True
initialize_photoZ = False
rewrite_data_files = True
skip_grid_drawing = True

In [3]:
# set up PhotoZ (open it manually)
aPhz = AutoPhotoZ(data_dir=data_dir)
if initialize_photoZ and not disable_photoZ_interact:
    aPhz.prepare_photoZ()

In [4]:

slice_count = 0
missing_la_file_count = 0
for subdir, dirs, files in os.walk(data_dir):
    if 'notUsable' in subdir:
        continue
    if 'selected_zda' not in subdir:
        continue
    print(subdir)
    subdir += '/'
    for filename in files:
        fn = filename.split(".")[0]
        if (fn.endswith("_snr") or fn.startswith("SNR")) and "roi" not in fn:
            name_parse = filename[:8]  # of the format 07-01-01
            if fn.startswith("SNR"):
                name_parse = filename[4:11]
            try:
                sep = "_"
                if "-" in name_parse:
                    sep = "-"
                slic, loc, rec = [int(x) for x in name_parse.split(sep)]
            except ValueError:
                print("Could not parse:", name_parse)
                continue
            if slic not in image_data:
                image_data[slic] = {}
            if loc not in image_data[slic]:
                image_data[slic][loc] = {}
            if rec not in image_data[slic][loc]:
                image_data[slic][loc][rec] = {}
            if subdir not in image_data[slic][loc][rec]:
                image_data[slic][loc][rec][subdir] = {}

            snr = np.loadtxt(subdir + filename,
                             delimiter='\t',
                             usecols=(1,))
            width = int(np.sqrt(snr.shape[0]))
            snr = snr.reshape((width, width))
            n_grid_size = int(width / n_sq)  # how many squares will fit along frame edge
            image_data[slic][loc][rec][subdir]['snr_map'] = snr
            print("slice", slic, ", rec", rec)
            
            if skip_grid_drawing:
                continue
            # corners file: 2 points p1, p2 to define the edge along which to measure
            la_file = subdir + corners_file_prefix_layer + '.dat'
            lines = None
            if not os.path.exists(la_file):
                missing_la_file_count += 1
            else:
                print("Processing", la_file)
                slice_count += 1
                with open(la_file, 'r') as f:
                    lines = f.readlines() 
                corners = [int(x) for x in lines[4:]] # the last 4 lines are diode numbers of corners
                layer_axes = LayerAxes(corners)
                laminar_axis, laminar_axis_2 = layer_axes.get_layer_axes()
                print("laminar axis unit vectors:", laminar_axis.get_unit_vector(), 
                      laminar_axis_2.get_unit_vector())

                # make our own square ROIs aligned to the list of edges and write them to a .dat file
                ################################
                # Limit # of ROIs by SNR to n highest SNR ROIs so that we can do
                #        smaller ROIs without exceeding 100-ROI-per-file cap
                ################################
                roi_cr = SquareROICreator(layer_axes, roi_width=n_sq, max_num_rois=max_num_rois, snr=snr)
                rois = roi_cr.get_rois()  # list of LaminarROI objects
                roi_centers = roi_cr.get_roi_centers(rounded=True) 
                # print("ROI centers:", roi_centers)
                roi_cr.write_roi_centers_to_file(subdir + "/" + rois_centers_filenames)

                # write these ROIs to file
                roi_fn = roi_cr.write_roi_file(subdir, rois_file_prefix)
                print("Created file:", roi_fn)

                # open stim point roi as a single integer (its diode number) in variable stim_pt
                sp_file = subdir + electrode_file_prefix + '.dat'
                with open(sp_file, 'r') as f:
                    lines = f.readlines()
                stim_pt = int(lines[-1]) # last line is always electrode location
                aux_obj = LaminarROI([stim_pt]).get_points()
                stim_pt = aux_obj[0]  # should be a list of len 2, representing px location [x, y]
                #print("Stim point:", stim_pt)

                # run laminar dist computation
                laminar_distances_obj = LaminarDistance(laminar_axis, rois, stim_pt)
                laminar_distances = laminar_distances_obj.compute_laminar_distances()
                laminar_distances = [round(ld, 2) for ld in laminar_distances]
                #print('laminar_distances:', laminar_distances)  # a list of integers with same indexing as rois

                # run laminar dist again with the other axis
                laminar_distances_2 = LaminarDistance(laminar_axis_2, rois, stim_pt).compute_laminar_distances()
                laminar_distances_2 = [round(ld, 2) for ld in laminar_distances_2]
                #print('laminar_distances 2:', laminar_distances_2)  # a list of integers with same indexing as rois

                # write averaged laminar distances to file
                avg_laminar_distances = [round(
                    (laminar_distances[i] + laminar_distances_2[i]) / 2,
                    2)
                                        for i in range(len(laminar_distances))]
                #print('laminar_distances averaged :', avg_laminar_distances) 
                ld_filename_base = la_file[:-4] + "_"
                laminar_distances_obj.write_laminar_distance_file(ld_filename_base, avg_laminar_distances)

                #image_data[slic][loc][rec][subdir] = {
                #    'snr': snr,
                #    'filename': filename}

                # Now that the data is stored, show intermediate computations in plot
                if n_plots > 0:
                    lines = [laminar_axis.get_line_repr(),
                             laminar_axis_2.get_line_repr()]
                    line_colors = ['red', 'yellow']
                    linewidths = [4,4]
                    other_colors = ['red', 'green', 'blue', 'white', 'purple']
                    roi_colors = []

                    # laminar aux lines
                    uv = laminar_axis.get_unit_vector()
                    for i in range(len(laminar_distances)):
                        ld = laminar_distances[i]
                        jiggle = i - int(len(laminar_distances) / 2)
                        uv_perp = Line(laminar_axis.get_start_point(), 
                                       laminar_axis_2.get_start_point()).get_unit_vector()
                        stim_proxy_pt = [stim_pt[0] - jiggle * uv_perp[0], 
                                         stim_pt[1] - jiggle * uv_perp[1]]
                        measure_vector = [uv[0] * ld + stim_proxy_pt[0], uv[1] * ld + stim_proxy_pt[1]]
                        lines.append([measure_vector, stim_proxy_pt])
                        this_color = other_colors[i % len(other_colors)]
                        line_colors.append(this_color)
                        roi_colors.append(this_color)
                        linewidths.append(1)

                    GridVisualization(snr, stim_pt, roi_centers, layer_axes.get_corners(),
                                      lines, line_colors, linewidths,
                                      rois, roi_colors,
                                      save_dir=subdir+"GridVisual1"+name_parse+'.png')
                    n_plots -= 1

print("\tslice directory count:", slice_count)
print("\tmissing_la_file_count:", missing_la_file_count)

	slice directory count: 0
	missing_la_file_count: 0


In [5]:
## 3) take SNRs and latencies to .dat file of each ROI from PhotoZ
slice_count = 0
missing_roi_file_count = 0
if not disable_photoZ_interact:
    for subdir, dirs, files in os.walk(data_dir):
        if 'notUsable' in subdir:
            continue
        if 'selected_zda' not in subdir:
            continue

        for zda_file in files:
            if zda_file.endswith('.zda'):
                rec_id = zda_file.split('.')[0]
                slic_id, loc_id, _ = [int(x) for x in rec_id.split("_")]
                print(rec_id)
                
                # check for existing files
                filename_end = "_rois_" + rec_id + ".dat"
                snr_filename = subdir + "/" + "snr_" + filename_end
                latency_filename = subdir + "/" + "latency_" + filename_end                
                need_to_open_zda = rewrite_data_files or ((not os.path.isfile(snr_filename)) 
                                        or (not os.path.isfile(latency_filename)))
                slice_count += 1
                
                if need_to_open_zda:
                    
                    roi_fn = rois_file_prefix + ".dat"
                    if roi_fn not in files:
                        print("\t", subdir, "has no file named", rois_file_prefix + ".dat") 
                        missing_roi_file_count += 1
                        continue
                    roi_fn = subdir + "/" + roi_fn
                    
                    # open this zda file in photoZ
                    aPhz = AutoPhotoZ(data_dir=subdir)
                    aPhz.select_PhotoZ()

                    print("\n\nOpening", zda_file)
                    aPhz.open_zda_file(subdir + "/" + zda_file)

                    # open ROI file
                    aPhz.select_roi_tab()
                    aPhz.open_roi_file(roi_fn)

                    # save SNR values
                    aPhz.select_SNR_trace_value()
                    aPhz.save_trace_values(snr_filename)

                    # save latency values
                    aPhz.select_latency_trace_value()
                    aPhz.save_trace_values(latency_filename)

                else:
                    print("snr_" + filename_end, "and", "latency_" + filename_end, "already exist in", subdir)

print("\tslice directory count:", slice_count)
print("\tmissing_roi_file_count:", missing_roi_file_count)

	slice directory count: 0
	missing_roi_file_count: 0


In [6]:
"""## filter out no-response grid squares by SNR cutoff (or manual inspection?)
###   Read in the SNR/latency files/roi center files
slice_count = 0
for slic in image_data:
    for loc in image_data[slic]:
        for rec in image_data[slic][loc]:
            for subdir in image_data[slic][loc][rec]:
                slice_count += 1
                def pad_z2(x):
                    x = str(x)
                    while len(x) < 2:
                        x = '0' + x
                    return x
                
                rec_id = pad_z2(slic) + "_" + pad_z2(loc) + "_" + pad_z2(rec)
                
                filename_end = "_rois_" + rec_id + ".dat"
                snr_filename = subdir + "/" + "SNR_" + filename_end
                latency_filename = subdir + "/" + "latency_" + filename_end 
                roi_center_filename = subdir + "/" + rois_centers_filenames


                snr_df = pd.read_csv(snr_filename, sep='\t', header=None,
                                     names=['Index',  'SNR'])
                latency_df = pd.read_csv(latency_filename, sep='\t', header=None,
                                     names=['ROI Index',  'Latency'])
                roi_center_df = pd.read_csv(roi_center_filename, sep='\t', header=None,
                                     names=['Center x',  'Center y'])
                
                combined_df = roi_center_df.join(latency_df)
                combined_df = combined_df.join(snr_df[['SNR']])
                print(subdir, rec_id)
                
                ############## Analysis: ############################
                ##### 1) directed graph analysis of latency #########
                ##### 2) 3-D plot of latency ########################
                ##### 3) 3-D plot of SNR ############################
                #####################################################
                
                # 1) directed graph analysis of latency
                nd_list = [Node([row[['Center x']][0], row[['Center y']][0]],
                                 row[['Latency']][0],
                                 row[['SNR']][0]) for _, row in combined_df.iterrows()]
                grid = Grid(nd_list, n_sq, latency_tolerance=latency_tolerance)
                
                ## a) determine latency ordering on directed graph
                grid.populate_latency_matrix()
                grid.calculate_current_field()  # current flow vectors of all nodes
                
                ## b) visualize
                snr_map = None
                try:
                    snr_map = image_data[slic][loc][rec][subdir]['snr_map']
                except Exception as e:
                    snr_map = np.zeros((80, 80))  # default
                grid.visualize_spatial_flow(snr_map=snr_map,
                                            save_dir=subdir+"GridVisual1"+name_parse+'.png')
                grid.visualize_current_field(snr_map=snr_map,
                                            save_dir=subdir+"GridVisual2"+name_parse+'.png',
                                            min_lat=0.1, 
                                            max_lat=3.0)
                
                
print("\tslice directory count:", slice_count) """

'## filter out no-response grid squares by SNR cutoff (or manual inspection?)\n###   Read in the SNR/latency files/roi center files\nslice_count = 0\nfor slic in image_data:\n    for loc in image_data[slic]:\n        for rec in image_data[slic][loc]:\n            for subdir in image_data[slic][loc][rec]:\n                slice_count += 1\n                def pad_z2(x):\n                    x = str(x)\n                    while len(x) < 2:\n                        x = \'0\' + x\n                    return x\n                \n                rec_id = pad_z2(slic) + "_" + pad_z2(loc) + "_" + pad_z2(rec)\n                \n                filename_end = "_rois_" + rec_id + ".dat"\n                snr_filename = subdir + "/" + "SNR_" + filename_end\n                latency_filename = subdir + "/" + "latency_" + filename_end \n                roi_center_filename = subdir + "/" + rois_centers_filenames\n\n\n                snr_df = pd.read_csv(snr_filename, sep=\'\t\', header=None,\n        